In [1]:
import pandas as pd

In [2]:
import sqlite3
conn=sqlite3.connect('my_data.db')

number of events that occur for each day for each channel

In [5]:
pd.read_sql('''select channel, strftime('%d',occurred_at), count(id) from web_events
group by 1,2''',conn)

,channel,"strftime('%d',occurred_at)",count(id)
0,adwords,01,36
1,adwords,02,33
2,adwords,03,25
3,adwords,04,25
4,adwords,05,29
...,...,...,...
181,twitter,27,17
182,twitter,28,19
183,twitter,29,10
184,twitter,30,12


sub query from 1st query

In [19]:
pd.read_sql('''select * from 
(
select channel, strftime('%d',occurred_at), count(id) from web_events
group by 1,2
) sub''',conn)

,channel,"strftime('%d',occurred_at)",count(id)
0,adwords,01,36
1,adwords,02,33
2,adwords,03,25
3,adwords,04,25
4,adwords,05,29
...,...,...,...
181,twitter,27,17
182,twitter,28,19
183,twitter,29,10
184,twitter,30,12


find the average number of events for each channel. Average per day

In [20]:
pd.read_sql('''select channel, avg(count_day) as count_day_channel  from 
(
select channel, strftime('%d',occurred_at) as day_event, count(id) as count_day from web_events
group by 1,2
) sub
group by channel''',conn)

,channel,count_day_channel
0,adwords,29.225806
1,banner,15.354839
2,direct,170.903226
3,facebook,31.193548
4,organic,30.709677
5,twitter,15.290323


list of orders happended at the first month in P&P history , ordered by occurred_at

In [46]:
pd.read_sql('''select *  from 
orders 
where strftime('%m',occurred_at)=
(select strftime('%m',min(occurred_at))
from orders)
and strftime('%Y',occurred_at)=
(select strftime('%Y',min(occurred_at))
from orders)
order by occurred_at''',conn)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,5786,2861,2013-12-04T04:22:44.000Z,NaN,48.0,33.0,81.0,0.00,359.52,267.96,627.48
1,2415,2861,2013-12-04T04:45:54.000Z,490.0,15.0,11.0,516.0,2445.10,112.35,89.32,2646.77
2,4108,4311,2013-12-04T04:53:25.000Z,528.0,10.0,NaN,538.0,2634.72,74.90,0.00,2709.62
3,4489,1281,2013-12-05T20:29:16.000Z,NaN,37.0,NaN,37.0,0.00,277.13,0.00,277.13
4,287,1281,2013-12-05T20:33:56.000Z,492.0,73.0,NaN,565.0,2455.08,546.77,0.00,3001.85
...,...,...,...,...,...,...,...,...,...,...,...
94,4756,1521,2013-12-31T00:18:22.000Z,10.0,60.0,29.0,99.0,49.90,449.40,235.48,734.78
95,702,1521,2013-12-31T00:36:59.000Z,266.0,NaN,26.0,292.0,1327.34,0.00,211.12,1538.46
96,3057,3371,2013-12-31T02:04:00.000Z,79.0,75.0,14.0,168.0,394.21,561.75,113.68,1069.64
97,2764,3101,2013-12-31T12:58:47.000Z,119.0,36.0,NaN,155.0,593.81,269.64,0.00,863.45


list of orders happended at the first day in P&P history , ordered by occurred_at

In [48]:
pd.read_sql('''select *  from 
orders 
where strftime('%d',occurred_at)=
(select strftime('%d',min(occurred_at))
from orders)
and strftime('%m',occurred_at)=
(select strftime('%m',min(occurred_at))
from orders)
and strftime('%Y',occurred_at)=
(select strftime('%Y',min(occurred_at))
from orders)
order by occurred_at''',conn)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,5786,2861,2013-12-04T04:22:44.000Z,NaN,48.0,33.0,81.0,0.00,359.52,267.96,627.48
1,2415,2861,2013-12-04T04:45:54.000Z,490.0,15.0,11.0,516.0,2445.10,112.35,89.32,2646.77
2,4108,4311,2013-12-04T04:53:25.000Z,528.0,10.0,NaN,538.0,2634.72,74.90,0.00,2709.62


average of paper quantity happended at the first month in P&P history

In [47]:
pd.read_sql('''select avg(total)  from 
orders 
where strftime('%m',occurred_at)=
(select strftime('%m',min(occurred_at))
from orders)
and strftime('%Y',occurred_at)=
(select strftime('%Y',min(occurred_at))
from orders)
order by occurred_at''',conn)

,avg(total)
0,588.989899


account id,name and its most frequenet used channel

In [62]:
pd.read_sql('''select web_events.account_id,accounts.name,web_events.channel from web_events
inner join 
(select 
channel,
count(*) from web_events
group by channel
order by 2 desc
limit 1) tab1
on web_events.channel=tab1.channel
inner join accounts
on web_events.account_id=accounts.id''',conn)

,account_id,name,channel
0,1001,Walmart,direct
1,1001,Walmart,direct
2,1001,Walmart,direct
3,1001,Walmart,direct
4,1001,Walmart,direct
...,...,...,...
5293,4491,PPL,direct
5294,4491,PPL,direct
5295,4491,PPL,direct
5296,4491,PPL,direct


sales rep total sales for each region

In [75]:
pd.read_sql('''select sales_reps.name,sum(orders.total_amt_usd),region.name from orders
join accounts on accounts.id=orders.account_id
join sales_reps on sales_reps.id=accounts.sales_rep_id
join region on region.id=sales_reps.region_id
group by sales_reps.name''',conn)

,name,sum(orders.total_amt_usd),name
0,Akilah Drinkard,136613.99,Northeast
1,Arica Stoltzfus,810353.34,West
2,Ayesha Monica,217146.59,Northeast
3,Babette Soukup,215905.27,Southeast
4,Brandie Riva,675917.64,West
5,Calvin Ollison,594516.08,Southeast
6,Cara Clarke,166138.65,Northeast
7,Carletta Kosinski,213032.45,Midwest
8,Charles Bidwell,675637.19,Midwest
9,Chau Rowles,184282.60,Midwest


maximum total sales in each region

In [93]:
pd.read_sql('''select max(orders.total_amt_usd) as total_region,region.name as region_name from orders
join accounts on accounts.id=orders.account_id
join sales_reps on sales_reps.id=accounts.sales_rep_id
join region on region.id=sales_reps.region_id
group by region_id''',conn)

,total_region,region_name
0,93106.81,Northeast
1,48675.90,Midwest
2,95005.82,Southeast
3,232207.07,West


Provide the name of the sales_rep in each region with the largest amount of total_amt_usd sales.

In [98]:
pd.read_sql('''select max(rep_table.total_rep),region_name from
(select sum(orders.total_amt_usd) as total_rep,region.name as region_name from orders
join accounts on accounts.id=orders.account_id
join sales_reps on sales_reps.id=accounts.sales_rep_id
join region on region.id=sales_reps.region_id
group by sales_reps.id) rep_table
group by region_name''',conn)

,max(rep_table.total_rep),region_name
0,675637.19,Midwest
1,1010690.60,Northeast
2,1098137.72,Southeast
3,886244.12,West


largest sales region

In [100]:
pd.read_sql('''select max(total_region),region_name from
(select sum(orders.total_amt_usd) as total_region,region.name as region_name from orders
join accounts on accounts.id=orders.account_id
join sales_reps on sales_reps.id=accounts.sales_rep_id
join region on region.id=sales_reps.region_id
group by region_id) tab_region''',conn)

,max(total_region),region_name
0,7744405.36,Northeast


total numbers of orders per region

In [103]:
pd.read_sql('''select count(orders.total_amt_usd) as total_region,region.name as region_name from orders
join accounts on accounts.id=orders.account_id
join sales_reps on sales_reps.id=accounts.sales_rep_id
join region on region.id=sales_reps.region_id
group by region_id''',conn)

,total_region,region_name
0,2357,Northeast
1,897,Midwest
2,2024,Southeast
3,1634,West


For the region with the largest (sum) of sales total_amt_usd, how many total (count) orders were placed?

In [107]:
pd.read_sql('''
select total_region,region_name from
(select count(orders.total_amt_usd) as total_region,region.name as region_name from orders
join accounts on accounts.id=orders.account_id
join sales_reps on sales_reps.id=accounts.sales_rep_id
join region on region.id=sales_reps.region_id
group by region_id)
where region_name= 
(select region_name from
(select max(total_region),region_name from
(select sum(orders.total_amt_usd) as total_region,region.name as region_name from orders
join accounts on accounts.id=orders.account_id
join sales_reps on sales_reps.id=accounts.sales_rep_id
join region on region.id=sales_reps.region_id
group by region_id) tab_region) max_region)''',conn)

,total_region,region_name
0,2357,Northeast


account with largest standard paper orders

In [118]:
pd.read_sql('''select tab1.accounts_name,max(tab1.total_amt) from
(select accounts.name as accounts_name,sum(orders.standard_amt_usd) as total_amt from accounts join
orders on orders.account_id=accounts.id
group by accounts.id) tab1''',conn)

,accounts_name,max(tab1.total_amt)
0,Core-Mark Holding,207668.83


max standard paper orders number

In [119]:
pd.read_sql('''select tab1.accounts_name,max(tab1.total_amt) from
(select accounts.name as accounts_name,count(orders.standard_amt_usd) as total_amt from accounts join
orders on orders.account_id=accounts.id
group by accounts.id) tab1''',conn)

,accounts_name,max(tab1.total_amt)
0,Leucadia National,71


For the customer that spent the most (in total over their lifetime as a customer) total_amt_usd, 
how many web_events did they have for each channel?

In [156]:
pd.read_sql('''select * from
(select account_id,channel,count(*) from web_events group by account_id,channel) tab1
join 
(select account_id,account_name,max(total_account) from
(select accounts.id as account_id,accounts.name as account_name,sum(orders.total_amt_usd) as total_account from accounts
join orders on orders.account_id=accounts.id
group by accounts.id)) tab2
on tab1.account_id=tab2.account_id''',conn)

,account_id,channel,count(*),account_id,account_name,max(total_account)
0,4211,adwords,12,4211,EOG Resources,382873.3
1,4211,banner,4,4211,EOG Resources,382873.3
2,4211,direct,44,4211,EOG Resources,382873.3
3,4211,facebook,11,4211,EOG Resources,382873.3
4,4211,organic,13,4211,EOG Resources,382873.3
5,4211,twitter,5,4211,EOG Resources,382873.3


What is the lifetime average amount spent in terms of total_amt_usd for the top 10 total spending accounts?

In [169]:
pd.read_sql('''select tab2.account_name,tab2.total_account/tab2.total_count from
(select account_id from web_events group by account_id) tab1
join 
(select accounts.id as account_id,accounts.name as account_name,sum(orders.total_amt_usd) as total_account,count(orders.total_amt_usd) as total_count from accounts
join orders on orders.account_id=accounts.id
group by accounts.id
order by total_account desc
limit 10) tab2
on tab1.account_id=tab2.account_id''',conn)

,account_name,tab2.total_account/tab2.total_count
0,EOG Resources,6175.375806
1,Mosaic,5236.645303
2,IBM,5446.991333
3,General Dynamics,4555.981667
4,Republic Services,5877.222800
5,Leucadia National,4099.257042
6,Arrow Electronics,4194.303881
7,Sysco,4096.700588
8,Supervalu,4048.357353
9,Archer Daniels Midland,4131.406667


What is the lifetime average amount spent in terms of total_amt_usd, including only the companies 
that spent more per order, on average, than the average of all orders

In [189]:
pd.read_sql('''select orders.account_id,avg(orders.total_amt_usd) from orders join
(select accounts.id as account_id,accounts.name as account_name,avg(orders.total_amt_usd) as avg_accounts from accounts
join orders on orders.account_id=accounts.id
group by accounts.id) tab2
on orders.account_id=tab2.account_id
where orders.total_amt_usd>tab2.avg_accounts
group by orders.account_id
''',conn)

,account_id,avg(orders.total_amt_usd)
0,1001,8704.446667
1,1021,2772.892857
2,1041,5011.060000
3,1051,2937.453333
4,1061,6161.036667
...,...,...
327,4461,6217.616800
328,4471,2278.160000
329,4481,3264.410000
330,4491,6609.870000


find the average number of events for each channel per day

In [229]:
pd.read_sql('''select tab1.channel, tab1.count_events/tab3.count_days  from
(select channel,cast(count(*) as decimal) as count_events from web_events group by channel) tab1
join
(select tab2.channel as channel,sum(tab2.count_per_year) as count_days from
(select channel,count(distinct strftime('%j',occurred_at)) as count_per_year,strftime('%Y',occurred_at) as year from web_events group by year,channel) tab2
group by tab2.channel) tab3
on tab1.channel=tab3.channel''',conn)

,channel,tab1.count_events/tab3.count_days
0,adwords,1
1,banner,1
2,direct,4
3,facebook,1
4,organic,1
5,twitter,1


Provide the name of the sales_rep in each region with the largest amount of total_amt_usd sales.

In [243]:
pd.read_sql('''select max(tab2.rep_total), tab2.region_name, tab2.sales_rep_name from
(select sales_reps.name as sales_rep_name,sum(orders.total_amt_usd) as rep_total,region.name as region_name from region
join sales_reps on sales_reps.region_id=region.id
join accounts on sales_reps.id=accounts.sales_rep_id
join orders on accounts.id=orders.account_id
group by sales_reps.id
order by rep_total desc) tab2
group by tab2.region_name''',conn)

,max(tab2.rep_total),region_name,sales_rep_name
0,675637.19,Midwest,Charles Bidwell
1,1010690.60,Northeast,Tia Amato
2,1098137.72,Southeast,Earlie Schleusner
3,886244.12,West,Georgianna Chisholm
